# Tetris 

COGS 18 Final Project: Tetris

Tetris is a tile-matching puzzle game first designed by Soviet Russian game designer Alexey Leonidovich Pajitnov.



## Based on this guide created by Professor David Kosbie


Author: Zijian Liu

PID: A15723355

Instructor: Professor Shannon Ellis

Spring 2019

I developed this implementation of Tetris using the starter code found here:

https://www.cs.cmu.edu/~112/notes/notes-tetris/index.html


## Project Code


In [27]:
from tkinter import *

import random

import Tetris


run(playTetris()[0]
,playTetris()[1])
# will print "Having a nice summer!" upon game closing

bye!
